In [1]:
import requests
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import base64

In [2]:
df = pd.read_csv(r'D:\tugas\mlops\proyek_pertama\data\data.csv')
df

,message,sentiment
0,I really enjoyed the movie,1
1,The food was terrible,0
2,I'm not sure how I feel about this,1
3,The service was excellent,1
4,I had a bad experience,0
...,...,...
579,I have to cancel my vacation plans because I c...,0
580,My computer crashed and I lost all my importan...,0
581,I got into a car accident and my car is totale...,0
582,I have a cold and can't stop coughing. it's re...,0


In [3]:
df['message']

0                             I really enjoyed the movie
1                                  The food was terrible
2                     I'm not sure how I feel about this
3                              The service was excellent
4                                 I had a bad experience
                             ...                        
579    I have to cancel my vacation plans because I c...
580    My computer crashed and I lost all my importan...
581    I got into a car accident and my car is totale...
582    I have a cold and can't stop coughing. it's re...
583    I just found out my ex is dating someone new. ...
Name: message, Length: 584, dtype: object

In [4]:
testing = df['message'][1]
testing

'The food was terrible'

## Make prediction request

In [5]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(testing)

endpoint = "http://localhost:8080/v1/models/chat_sentiment-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Positive" if prediction_value > 0.75 else "Negative"
else:
    result = "Error: No predictions found in response."

print(result)

Error: No predictions found in response.
